### Problem Statement

#### Objectives
As a new hire to the ML Engineering team at CyberProtect, you have been entrusted with a critical task: protecting the users of CyberProtect from phishing attacks when they are assessing websites on the internet.

CyberProtect has collected a database of phishing websites and legitimate websites. CyberProtect’s data engineering team has extracted various features from these websites that may be helpful to your tasks. These features are described below in the “List of Attributes” section.

By leveraging these features, you will develop predictive models that can be installed as an extension and predict if a website is a phishing attack and warn the user before allowing the user to access it.

Specifically, your task is to build and evaluate prediction models, and also identify their respective key features of the dataset that categorise whether the website is a phishing attack. Your analysis should also include evaluation of which features contribute most significantly to phishing attack prediction.

In your submission, you are expected to build and evaluate at least three suitable models for this task and justify your choices based on the dataset provided.

#### Dataset
The dataset contains various features such as number of URL redirects, number of external references , which are gathered by the data engineering team to help your work. Specific information on the various dataset features shall be provided in the next page.

Important Note: The dataset contains synthetic or contaminated data. Therefore, you should state clearly any assumptions or justifications that you make in processing the data, including handling of outliers, missing values, and data quality issues.

You can query the dataset using the following URL:
https://techassessment.blob.core.windows.net/aiap22-assessment-data/phishing.db

#### List of Attributes
| Attribute           | Description                                               |
|---------------------|-----------------------------------------------------------|
| LineOfCode          | Number of lines of code                                  |
| LargestLineLength   | Longest line of code                                     |
| NoOfURLRedirect     | Number of URL redirects from the website                 |
| NoOfSelfRedirect    | Number of self-redirects from the website                |
| NoOfPopup           | Number of pop-ups from the website                       |
| NoOfiFrame          | Number of iFrames from the website                       |
| NoOfSelfRef         | Number of clickable links within the same domain         |
| NoOfExternalRef     | Number of clickable links to external websites           |
| Robots              | Whether the website has a robots.txt                     |
| IsResponsive        | Whether the website adapts appropriately across devices  |
| Industry            | The industry the website belongs to                      |
| DomainAgeMonths     | Months since the domain was created                      |
| HostingProvider     | The hosting provider of the website                      |
| label               | 0 = phishing website, 1 = legitimate website             |

Theres an additional attribute not listed in the PDF: NoOfImages

### Library / Dependencies

In [3]:
# Import Libraries / Dependencies
import sqlite3
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np

# Analysis and Transformers
#from scipy.stats import ttest_ind, ks_2samp, chi2_contingency
#from collections import defaultdict
#from sklearn.preprocessing import OneHotEncoder,  StandardScaler , MinMaxScaler , LabelEncoder
#from sklearn.model_selection import train_test_split
#from sklearn.feature_selection import mutual_info_classif # ,mutual_info_regression
#from sklearn.impute import KNNImputer

### Data Cleaning

#### Data Import

In [6]:
# Import Raw Data
# Connect to DB
conn = sqlite3.connect('data/phishing.db')
# Get list of all tables
table_query = "SELECT name FROM sqlite_master WHERE type='table';"
tables = pd.read_sql(table_query, conn)
# Load all tables into a dictionary of dataframes
dataframes = {}
for table_name in tables['name']:
    df = pd.read_sql(f"SELECT * FROM {table_name}", conn)
    dataframes[table_name] = df
    print(f"Loaded table: {table_name}, shape: {df.shape}")

Loaded table: phishing_data, shape: (10500, 16)


In [7]:
# Querying the table
query = f"SELECT * FROM phishing_data"
df_raw = pd.read_sql_query(query, conn)
conn.close() # Good Practice to close
df_raw.head(10)

,Unnamed: 0,LineOfCode,LargestLineLength,NoOfURLRedirect,NoOfSelfRedirect,NoOfPopup,NoOfiFrame,NoOfImage,NoOfSelfRef,NoOfExternalRef,Robots,IsResponsive,Industry,HostingProvider,DomainAgeMonths,label
0,0,NaN,9965,0,0,2,1,48,172,170,1,1,Fashion,DreamHost,98,0
1,1,769.0,9507,0,0,0,4,25,55,78,1,1,Fashion,AWS,31,1
2,2,720.0,2550,0,0,0,2,-31,94,13,0,1,Education,Google Cloud,26,1
3,3,198.0,869,0,0,0,0,0,0,2,0,0,Unknown,DreamHost,2,1
4,4,972.0,2807,0,0,0,2,6,83,51,0,1,Education,GoDaddy,43,1
5,5,247.0,4462,0,0,0,0,3,4,4,0,1,Manufacturing,AWS,8,1
6,6,2.0,80,0,0,0,0,0,0,0,0,0,Unknown,Freehostia,8,0
7,7,452.0,9381,0,0,0,0,2,36,42,1,1,Fashion,Bluehost,42,1
8,8,1018.0,9381,0,0,0,2,20,95,96,1,1,Manufacturing,Google Cloud,30,0
9,9,418.0,120391,0,0,0,3,2,0,2,1,0,eCommerce,DreamHost,2,0


In [9]:
# it is good practice to leave raw data untouched, so make a copy for analysis and revision
df_data = df_raw.copy()

#### Cursory Examination

In [10]:
# Quick examination of the data
df_data.nunique()

Unnamed: 0           10500
LineOfCode            2875
LargestLineLength     4027
NoOfURLRedirect          2
NoOfSelfRedirect         2
NoOfPopup               26
NoOfiFrame              48
NoOfImage              308
NoOfSelfRef            533
NoOfExternalRef        463
Robots                   2
IsResponsive             2
Industry                11
HostingProvider         13
DomainAgeMonths        121
label                    2
dtype: int64

In [11]:
# Quick examination of null values and dtypes
df_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10500 entries, 0 to 10499
Data columns (total 16 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   Unnamed: 0         10500 non-null  int64  
 1   LineOfCode         8145 non-null   float64
 2   LargestLineLength  10500 non-null  int64  
 3   NoOfURLRedirect    10500 non-null  int64  
 4   NoOfSelfRedirect   10500 non-null  int64  
 5   NoOfPopup          10500 non-null  int64  
 6   NoOfiFrame         10500 non-null  int64  
 7   NoOfImage          10500 non-null  int64  
 8   NoOfSelfRef        10500 non-null  int64  
 9   NoOfExternalRef    10500 non-null  int64  
 10  Robots             10500 non-null  int64  
 11  IsResponsive       10500 non-null  int64  
 12  Industry           10500 non-null  object 
 13  HostingProvider    10500 non-null  object 
 14  DomainAgeMonths    10500 non-null  int64  
 15  label              10500 non-null  int64  
dtypes: float64(1), int64(1

#### Duplicate data in the set

In [52]:
# Checking Duplciates
df_duplicated = df_data[df_data.duplicated(keep=False)]
df_duplicated.nunique()

LineOfCode           10
LargestLineLength    20
NoOfURLRedirect       1
NoOfSelfRedirect      1
NoOfPopup             1
NoOfiFrame            3
NoOfImage             4
NoOfSelfRef           5
NoOfExternalRef       6
Robots                2
IsResponsive          2
Industry              7
HostingProvider       9
DomainAgeMonths      21
label                 2
HasURLRedirect        1
HasSelfRedirect       1
dtype: int64

In [53]:
# Examining missing hosting providers
df_duplicated['HostingProvider'].unique()

array(['InfinityFree', 'Namecheap', '000webhost', 'DreamHost', 'Bluehost',
       'Hostinger', 'Unknown Provider', 'Freehostia', 'HostGator'],
      dtype=object)

`NoOfURLRedirect`, `NoOfSelfRedirect`, `NoOfPopup` all have the same outputs.  
Azure, Godaddy, AWS, Google Cloud are not amongst the hosts.  


##### Cursory Assessment  
Null values for `LineOfCode`. The dtype should be int instead of float.  
string cleaning might be needed for object dtypes.  
Unamed: 0 appears to be another index and can be dropped.  
There is a attribute not listed in the initial table `NoOfImage`, it as been added post assessment.

#### Dropping ghost-index `Unnamed: 0`

In [16]:
# Checking for unique counts
df_data['Unnamed: 0'].nunique() == len(df_data)

True

In [19]:
# checking for Total value difference
(df_data['Unnamed: 0'] - df_data.index).abs().sum()

KeyError: 'Unnamed: 0'

In [18]:
# dropping the column
df_data = df_data.drop(columns=['Unnamed: 0'])

In [21]:
# Quick Confirmation
df_data.head()

,LineOfCode,LargestLineLength,NoOfURLRedirect,NoOfSelfRedirect,NoOfPopup,NoOfiFrame,NoOfImage,NoOfSelfRef,NoOfExternalRef,Robots,IsResponsive,Industry,HostingProvider,DomainAgeMonths,label
0,NaN,9965,0,0,2,1,48,172,170,1,1,Fashion,DreamHost,98,0
1,769.0,9507,0,0,0,4,25,55,78,1,1,Fashion,AWS,31,1
2,720.0,2550,0,0,0,2,-31,94,13,0,1,Education,Google Cloud,26,1
3,198.0,869,0,0,0,0,0,0,2,0,0,Unknown,DreamHost,2,1
4,972.0,2807,0,0,0,2,6,83,51,0,1,Education,GoDaddy,43,1


#### Rename`IsResponsive` to `is_responsive` 

#### Rename`Robots` to `has_robots.txt`

In [ ]:
# Renaming to make features clearer, dtype conversion for memory saving

#### Rename `label` to `is_legitimate`

In [ ]:
# Renaming to make features clearer, dtype conversion for memory saving
df.data(columns={"label": "is_legitimate"}, inplace=True)

#### Examining `Industry` and `hostingprovider`

In [37]:
df_data['Industry'].unique()

array(['Fashion', 'Education', 'Unknown', 'Manufacturing', 'eCommerce ',
       'Non-profit', 'Food', 'Government', 'eCommerce', 'Banking',
       'Healthcare'], dtype=object)

In [38]:
df_data['HostingProvider'].unique()

array(['DreamHost', 'AWS', 'Google Cloud', 'GoDaddy', 'Freehostia',
       'Bluehost', 'Unknown Provider', 'Hostinger', 'Namecheap',
       'InfinityFree', '000webhost', 'Azure', 'HostGator'], dtype=object)

#### Examining `LineOfCode`'s dtype, Converting to `int64`

In [27]:
# Checking if there are fractional values
df_fractional = df_data[df_data['LineOfCode'] % 1 != 0].copy()
df_fractional['LineOfCode'].sum()

np.float64(0.0)

In [29]:
# Changing the dtype as its supposed to be semanticly discrete
df_data['LineOfCode'] = df_data['LineOfCode'].astype('Int64')
df_data['LineOfCode'].info()

<class 'pandas.core.series.Series'>
RangeIndex: 10500 entries, 0 to 10499
Series name: LineOfCode
Non-Null Count  Dtype
--------------  -----
8145 non-null   Int64
dtypes: Int64(1)
memory usage: 92.4 KB


#### Examining `LineOfCode`'s NaN Values

In [30]:
# Examining number of NaN Values
df_data['LineOfCode'].isna().sum()

np.int64(2355)

These values constitute 22.4% of the dataset and constitutes a large part of the set. We should endeavour investigate and to preserve it.

In [22]:
# Isolating NaN rows
df_nan = df_data[df_data['LineOfCode'].isna()].copy()

In [23]:
df_nan.head()

,LineOfCode,LargestLineLength,NoOfURLRedirect,NoOfSelfRedirect,NoOfPopup,NoOfiFrame,NoOfImage,NoOfSelfRef,NoOfExternalRef,Robots,IsResponsive,Industry,HostingProvider,DomainAgeMonths,label
0,NaN,9965,0,0,2,1,48,172,170,1,1,Fashion,DreamHost,98,0
14,NaN,11876,0,0,0,1,16,45,54,1,1,Government,AWS,15,0
15,NaN,394,0,0,0,0,0,2,1,1,1,Education,Freehostia,11,0
20,NaN,212,1,0,0,0,16,0,2,0,1,eCommerce,InfinityFree,5,0
22,NaN,105,1,1,0,0,0,1,1,0,0,Unknown,Unknown Provider,47,0


### Feature Engineering 

#### Feature relation to phishing

A rough understandinf of the attributes/features will help with effective feature engineering

| Attribute           | Description                                               | Possible Attack Vector / Threat Signal                                   |
|---------------------|-----------------------------------------------------------|---------------------------------------------------------------------------|
| LineOfCode          | Number of lines in the HTML/JS/CSS source                | Very small codebases may indicate *single-page phishing kits*; extremely long minified lines can hide *obfuscated payloads*. |
| LargestLineLength   | Length of the longest line of source                     | Long single-line JS often used for *obfuscation*, *malicious script injection*, or *form hijacking*. |
| NoOfURLRedirect     | Total number of HTTP redirects                           | *Redirect chains* used to hide malicious origins, rotate landing pages, track victims, or evade detection. |
| NoOfSelfRedirect    | Redirects back to same domain                            | Masked redirection surfaces; can support *open redirect setups* or hide spoofed URLs. |
| NoOfPopup           | Count of pop-up windows                                 | Used in *fake alerts*, *support scams*, credential prompts, or pressure-inducing dialogs. |
| NoOfiFrame          | Count of iframe elements                                 | Iframes can host *cloaked phishing pages*, *credential-stealing forms*, or *drive-by scripts*. |
| NoOfImage           | Number of `<img>` elements in the HTML                   | Excessive or stolen brand images; hidden tracking pixels; image-based phishing layouts |
| NoOfSelfRef         | Internal links within same domain                        | Low internal structure suggests *throwaway phishing pages* with no real site architecture. |
| NoOfExternalRef     | Outbound links to other domains                          | Few/no external links may indicate isolation typical of *phishing kits*; unusual external links may hide *malicious third-party scripts*. |
| Robots              | Presence of `robots.txt`                                 | Lack of robots.txt common in *temporary malicious domains* that avoid SEO and indexing. |
| IsResponsive        | Whether layout adapts across devices                     | Broken mobile responsiveness typical of *template-based phishing kits* not optimised for real use. |
| Industry            | Inferred industry of site                                | Attackers often mimic *finance*, *government*, *healthcare* for targeted credential harvesting. |
| DomainAgeMonths     | Age of domain                                            | Very new domains strongly tied to *domain churn*, common in phishing and malware delivery. |
| HostingProvider     | Hosting provider name                                    | Bulletproof or low-reputation hosts associated with *malware hosting* or *phishing kits*. |
| label               | Ground truth: 0 = phishing, 1 = legitimate               | — |



#### Binarisation of `NoOfURLRedirect` and `NoOfSelfRedirect` to create `HasURLRedirect` and `HasSelfRedirect`

The features `NoOfURLRedirect` and `NoOfSelfRedirect` currently hold exclusively binary values (0 or 1) in the phishing.db dataset, despite their names suggesting a numerical count.  
This discrepancy poses a risk if new datasets are introduced where these counts are greater than 1. 

To mitigate this risk and enhance model performance, we apply Binarization to create two new features.  
This engineering step achieves three primary goals:  

- Captures Non-linearity:
Creates a boolean flag (0 or 1) that explicitly isolates the highly predictive 'presence effect'—the difference between zero (no redirects) and non-zero (at least one redirect)—which is often more significant than the difference between count values (e.g., 1 vs. 2).

- Future-Proofing:
Provides a stable, binary signal for the model that remains distinct from the original count features when their values increase past one in new data.

- Interpretability:
Makes the model easier to interpret by allowing direct measurement of the impact of the simple presence of a redirect event.

In [35]:
# Create new features
df_data['HasURLRedirect'] = (df_data['NoOfURLRedirect'] > 0).astype(int)
df_data['HasSelfRedirect'] = (df_data['NoOfSelfRedirect'] > 0).astype(int)